In [3]:
# Notebook 2: Validate & Preprocess for T5
# !pip install transformers sqlparse

In [4]:
import pandas as pd
import sqlparse
from transformers import AutoTokenizer
import numpy as np

In [6]:
# Load
train_df = pd.read_pickle('train_raw.pkl')
test_df = pd.read_pickle('test_raw.pkl')

# SQL validation
def validate_sql(sql):
    try:
        parsed = sqlparse.parse(sql)
        return len(parsed) > 0 and len(parsed[0].tokens) > 0
    except:
        return False

print("Validating SQL...")
train_df = train_df.dropna(subset=['sql_prompt', 'sql'])

# Create a boolean column first, then filter
train_df['sql_valid'] = train_df['sql'].apply(validate_sql)
train_df = train_df[train_df['sql_valid'] == True].drop(columns=['sql_valid'])

test_df = test_df.dropna(subset=['sql_prompt', 'sql'])
test_df['sql_valid'] = test_df['sql'].apply(validate_sql)
test_df = test_df[test_df['sql_valid'] == True].drop(columns=['sql_valid'])

print(f"After validation - Train: {len(train_df)}, Test: {len(test_df)}")

Validating SQL...
After validation - Train: 100000, Test: 5851


In [7]:
# T5 formatting
def format_for_t5(row):
    if pd.notna(row['sql_context']) and row['sql_context'].strip():
        return f"translate English to SQL: {row['sql_prompt']} context: {row['sql_context']}"
    return f"translate English to SQL: {row['sql_prompt']}"

def clean_sql(sql):
    try:
        return sqlparse.format(sql, reindent=False, keyword_case='upper', strip_comments=True).strip()
    except:
        return ' '.join(sql.split())  # Fallback to simple cleaning

train_df['input_text'] = train_df.apply(format_for_t5, axis=1)
train_df['target_text'] = train_df['sql'].apply(clean_sql)
test_df['input_text'] = test_df.apply(format_for_t5, axis=1)
test_df['target_text'] = test_df['sql'].apply(clean_sql)

# Token analysis
tokenizer = AutoTokenizer.from_pretrained("gaussalgo/T5-LM-Large-text2sql-spider")
sample = train_df.sample(min(1000, len(train_df)))

input_lens = [len(tokenizer.encode(text)) for text in sample['input_text']]
output_lens = [len(tokenizer.encode(text)) for text in sample['target_text']]

print(f"\nToken stats:")
print(f"Input: mean={np.mean(input_lens):.0f}, max={max(input_lens)}, >512={sum(l>512 for l in input_lens)}")
print(f"Output: mean={np.mean(output_lens):.0f}, max={max(output_lens)}, >512={sum(l>512 for l in output_lens)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]


Token stats:
Input: mean=142, max=420, >512=0
Output: mean=53, max=263, >512=0


In [8]:
# Save
train_df[['input_text', 'target_text', 'domain', 'sql_complexity']].to_pickle('train_processed.pkl')
test_df[['input_text', 'target_text', 'domain', 'sql_complexity']].to_pickle('test_processed.pkl')
print("Preprocessed and saved")

Preprocessed and saved
